In [ ]:
import numpy as np

import astropy.units as u  
import astropy.constants as c
from astropy.coordinates import SkyCoord, Galactic, CartesianRepresentation

import matplotlib.pyplot as plt
import matplotlib 


from itertools import product

from astropy.io import fits
from astropy.table import Table, join

import sys
import os
import tqdm

bayestar_path = '/uufs/astro.utah.edu/common/home/u1371365/dustmaps_data/bayestar/bayestar2019.h5'
from dustmaps.bayestar import BayestarQuery

### NEW 03-20: implement the 10pc resolution Vergely map into dustmaps ###
from dustmaps.vergely2022 import Vergely2022Query

import h5py
from dustapprox.models import PrecomputedModel

import emcee

In [ ]:
distance = np.linspace(0, 1000, 200)


l0, b0 = (163., -8.0)
l_ = np.linspace(l0 - 9., l0 + 9., 200)
b_ = np.linspace(b0 - 9., b0 + 9., 200)
l, b, d = np.meshgrid(l_, b_, distance, indexing = 'xy') # remember that this switches the first two axes
print(l.shape)

coords = SkyCoord(l = l*u.deg, b = b*u.deg,
                  distance=distance*u.pc, frame='galactic')

In [ ]:
### Get Vergely+ (2022) dustmap for same points as Bayestar, now that it's in dustmaps
A0Coeff = 1.052180128669157 # from 02-03dustmap_explore.ipynb, calculated via dustapprox

vergelyquery = Vergely2022Query(map_fname = 
                                '/uufs/astro.utah.edu/common/home/u1371365/dustmaps_data/vergely2022/vergely22_extinction_density_resol_010pc.h5')
vergely = vergelyquery(coords) * A0Coeff
verg_reddening = np.cumsum(vergely, axis = 2) * 5

bayestarquery = BayestarQuery(map_fname = bayestar_path)
bayestar = bayestarquery(coords) * 3.1



fig, ax = plt.subplots(figsize = (6,6))
ax.imshow(np.sum(vergely * 5, axis = 2), origin = 'lower', cmap = 'binary', extent = (l0-9, l0+9, b0-9, b0+9))
ax.set_xlabel('l (deg)')
ax.set_ylabel('b (deg)')
plt.show()

fig, ax = plt.subplots(figsize = (6,6))
ax.imshow(bayestar[:, :, -1], origin = 'lower', cmap = 'binary', extent = (l0-9, l0+9, b0-9, b0+9))
ax.set_xlabel('l (deg)')
ax.set_ylabel('b (deg)')
plt.show()

In [ ]:
CAresdir = '/uufs/astro.utah.edu/common/home/u1371365/StellarResidualsSpring2022/CACloudResiduals/'
respath  = '/uufs/astro.utah.edu/common/home/u1371365/StellarResidualsSpring2022/Residuals/'

CAresdir = '/uufs/astro.utah.edu/common/home/u1371365/Data/20230420_CAResiduals/'
CA_meta = Table(fits.open(CAresdir + 'CA_meta.fits')[1].data)

tabfile = '/uufs/chpc.utah.edu/common/home/sdss/dr17/apogee/spectro/aspcap/dr17/synspec_rev1/allStar-dr17-synspec_rev1.fits'
stars = Table.read(tabfile, hdu = 1)

stars_addon = stars['APOGEE_ID', 'SFD_EBV', 'SNR', 'ASPCAPFLAG', 'STARFLAG', 'ASPCAP_CHI2']
CA_meta['DIST'] = 1000 / CA_meta['GAIAEDR3_PARALLAX']

CA_meta = join(CA_meta, stars_addon, keys = 'APOGEE_ID', join_type = 'left',)

CA_meta = CA_meta[(CA_meta['DIST'] > 250) & (CA_meta['DIST'] < 800)]

starhorsepath = '/uufs/chpc.utah.edu/common/home/sdss/dr17/env/APOGEE_STARHORSE/APOGEE_DR17_EDR3_STARHORSE_v2.fits'
starhorse = Table.read(starhorsepath, hdu = 1)
starhorse = starhorse['APOGEE_ID', 'dist16', 'dist50', 'dist84', 'AV16', 'AV50', 'AV84']

CA_meta = join(CA_meta, starhorse, keys = 'APOGEE_ID', join_type = 'left')

In [ ]:
CA_meta

In [ ]:
star_coords = SkyCoord(l = CA_meta['GLON'] * u.deg, b = CA_meta['GLAT'] * u.deg, distance = CA_meta['DIST'] * u.pc, frame = 'galactic')
verg_star = vergelyquery(star_coords)
# bayestar_star = bayestar()

In [ ]:
verg_array = np.zeros(len(CA_meta))
for i, star in enumerate(CA_meta):
    star_coord = SkyCoord(star['GLON'] * u.deg, star['GLAT'] * u.deg, distance = np.arange(0, star['DIST'] + 5, 5) * u.pc, frame = 'galactic')
    verg_star = vergelyquery(star_coord)
    verg_array[i] = np.sum(verg_star) * 5

In [ ]:
bayestar_array = bayestarquery(star_coords) * 3.1

In [ ]:
lambda0 = 15272.42 
sigma0 = 1.37

def get_wavs(hdulist = None, rv = 0):
    if hdulist is None:
        CRVAL1 = 4.179
        CDELT1 = 6e-06
        LEN = 8575
    else:
        header = hdulist[1].header
        CRVAL1 = header['CRVAL1']
        CDELT1 = header['CDELT1']
        LEN = header['NAXIS1']
        
    wavs = np.power(10, CRVAL1 + CDELT1 * np.arange(LEN))
    wavs = wavs * (1 + rv / 3e5) # allows for shifting to observed frame from rest frame
    return wavs

wavs = get_wavs()
window = (wavs > lambda0 -10) & (wavs < lambda0 + 10)
wavs_window = wavs[window]
window_mask = (wavs_window < lambda0) - 5 | (wavs_window > lambda0 + 5)

def dopplershift(v, lambda0 = lambda0):
     return (lambda0 * u.Angstrom * (c.c + v * u.km / u.s) / c.c).to(u.Angstrom).value

In [ ]:
def get_ca_res(fname):
    return str(CAresdir + str(fname))

for i in range(100):
    asdf = get_ca_res(CA_meta[i]['FILE'])
    try:
        fits.open(asdf)
    except:
        print('file doesnt exist')
        

def select_stars(tab, l0, b0, radius = 1):
    cond = np.sqrt((tab['GLON'] - l0)**2 + (tab['GLAT'] - b0)**2) < radius
    return np.where(cond)[0]

In [ ]:
def find_nearest(ll, bb):
    l_sel = l_
    b_sel = b_

    return np.argmin(np.abs(l_sel - ll)), np.argmin(np.abs(b_sel - bb))

def find_nearest_dist(d):
    return np.argmin(np.abs(distance[:, np.newaxis] - d), axis = 0)


def dAV_dd(l0, b0, bin_edges):
    l_ind, b_ind = find_nearest(l0, b0)
    sightline = np.copy(vergely[l_ind, b_ind, :])

    d_min, d_max = bin_edges

    extinction = sightline[(distance > d_min) & (distance < d_max)]
    return np.sum(extinction )

def dAV_dd_star(l0, b0, bin_edges, distances):
    l_ind, b_ind = find_nearest(l0, b0)
    d_min, d_max = bin_edges
    sightline = np.copy(vergely[l_ind, b_ind, :])
    sightline[(distance < d_min) | (distance > d_max)] = 0
    sightline_av = (np.cumsum(sightline))
    d_ind = find_nearest_dist(distances)

    return np.nanmedian(sightline_av[d_ind])

In [ ]:
res_array = np.zeros((len(CA_meta), len(wavs_window)))
dustmaps_inds = np.zeros((len(CA_meta), 3)).astype(int)

for i in range(len(CA_meta)):
    star = CA_meta[i]
    try:
        res_hdu = fits.open(get_ca_res(star['FILE']))
        res_array[i, :] = res_hdu[1].data[window]
    except:
        res_array[i, :] = np.nan

    try:
        l_ind, b_ind = find_nearest(star['GLON'], star['GLAT'])
        d_ind = find_nearest_dist(star['DIST'])
        dustmaps_inds[i, :] = np.array((l_ind, b_ind, d_ind)) #remember that the two indices switch
    except:
        dustmaps_inds[i, :] = 0


In [ ]:
CA_meta['Dustmap_AV'] = verg_reddening[dustmaps_inds[:,1], dustmaps_inds[:, 0], dustmaps_inds[:, 2]]

In [ ]:
dustmaps_inds

In [ ]:
fig, axs = plt.subplots(nrows = 1, ncols = 2, sharey = True, figsize = (16, 6))
axs[0].scatter(CA_meta['DIST'], CA_meta['Dustmap_AV'])
axs[0].set_xlabel('Distance (pc)')
axs[0].set_ylabel('Vergely Dustmap AV')
axs[1].scatter(CA_meta['DIST'], CA_meta['VERG_AV'])
axs[1].set_xlabel('Distance (pc)')
axs[1].set_ylabel('Vergely G-TOMO AV')
axs[0].set_ylim(0, 2.5)

plt.show()

fig, axs = plt.subplots(nrows = 1, ncols = 2, sharey = True, figsize = (16, 6))
axs[0].scatter(CA_meta['DIST'], CA_meta['AV50'])
axs[0].set_xlabel('Distance (pc)')
axs[0].set_ylabel('StarHorse AV')
axs[1].scatter(CA_meta['DIST'], CA_meta['AK_TARG'] / 0.302 * 2.742)
axs[1].set_xlabel('Distance (pc)')
axs[1].set_ylabel('AK_TARG AV')
axs[0].set_ylim(0, 2.5)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (6,6))
ax.scatter(CA_meta['VERG_AV'], CA_meta['Dustmap_AV'], alpha = 0.6)
ax.set_xlabel('Vergely G-TOMO AV')
ax.set_ylabel('Vergely Integrated AV')
ax.set_xlim(0, 3)
ax.set_ylim(0, 3)
ax.set_aspect('equal')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (6,6))
ax.scatter(CA_meta['AV50'], CA_meta['VERG_AV'], alpha = 0.6)
ax.set_xlabel('StarHorse AV')
ax.set_ylabel('Vergely G-TOMO AV')
ax.set_aspect('equal')
ax.set_xlim(0, 3)
ax.set_ylim(0, 3)
ax.set_aspect('equal')
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize = (24, 6))
axs[0].scatter(CA_meta['AV50'], CA_meta['Dustmap_AV'], alpha = 0.5)
axs[0].set_xlabel('Starhorse AV50')
axs[0].set_ylabel('Vergely AV')
axs[0].plot([0,3], [0,3], linestyle = 'dashed')
axs[0].set_aspect('equal')
axs[0].set_xlim(0, 3)
axs[0].set_ylim(0,3)

axs[1].scatter(CA_meta['AK_TARG'] / 0.302 * 2.742, CA_meta['Dustmap_AV'], alpha = 0.5)
axs[1].set_xlim(0, 3)
axs[1].set_ylim(0,3)
axs[1].set_xlabel('AK_TARG')
axs[1].set_ylabel('Vergely AV')
axs[1].plot([0,3], [0,3], linestyle = 'dashed')
axs[1].set_aspect('equal')

axs[2].scatter(CA_meta['AK_TARG'] / 0.302 * 2.742, CA_meta['AV50'], alpha = 0.5)
axs[2].set_xlim(0, 3)
axs[2].set_ylim(0,3)
axs[2].set_xlabel('AK_TARG')
axs[2].set_ylabel('Starhorse AV')
axs[2].plot([0,3], [0,3], linestyle = 'dashed')
axs[2].set_aspect('equal')
fig.set_facecolor('white')
plt.show()

In [ ]:
# Make plot of DIB EQW (gaussian or integ) vs
# STARHORSE
# AK_TARG
# VERGELY
# BAYESTAR

In [ ]:
def model(x, mu, sig, A, m, b ):
    return m * x + b - A * np.exp(-(x - mu)**2 / (2 * sig**2))

def line(x, m, b):
    return m * x + b

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
# wavs_window_repeat = np.repeat(wavs_window, len(res_array)).reshape(len(res_array), -1)
fit_params = np.zeros((len(CA_meta), 5, 2))
for i in range(len(CA_meta)):
    try:
        fit_result = curve_fit(model, wavs_window, res_array[i, :], p0 = (lambda0, 1.5, 0.05, 0, 1), bounds = ((lambda0-4, 0.5, 0, -.05, .95), (lambda0+4,  3,   .15,  .05, 1.05)), check_finite  = False)
        fit_params[i, :, 0] = fit_result[0]
        fit_params[i, :, 1] = np.sqrt(np.diag(fit_result[1]))


    except:
        fit_params[i, : , 0] = np.nan
        fit_params[i, : , 1] = np.nan

In [ ]:
AV = np.array(CA_meta['Dustmap_AV'])
amplitude = fit_params[:, 2, 0]
amplitude_errs = fit_params[:, 2, 1]
AV.shape == amplitude.shape

In [ ]:

fig, ax = plt.subplots(figsize = (8,8))
ax.imshow(np.sum(vergely * 5, axis = 2), origin = 'lower', cmap = 'binary', extent = (l0-9, l0+9, b0-9, b0+9))
ax.set_xlabel('l (deg)')
ax.set_ylabel('b (deg)')
ax.set_xlim(158, 168)
ax.set_ylim(-13, -4)

norm = matplotlib.colors.Normalize(-2, 2)
points = ax.scatter(CA_meta['GLON'], CA_meta['GLAT'], c= CA_meta['VERG_AV'] - CA_meta['Dustmap_AV'], s = 12, cmap = 'coolwarm', norm = norm)
fig.colorbar(points, label = 'G-TOMO -Integrated AV', )
fig.set_facecolor('white')

plt.show()

#### 
fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (16,8))
# axs[0].imshow(np.sum(vergely * 5, axis = 2), origin = 'lower', cmap = 'binary', extent = (l0-9, l0+9, b0-9, b0+9))
axs[0].imshow(bayestar[:, :, -1], origin = 'lower', cmap = 'binary', extent = (l0-9, l0+9, b0-9, b0+9))

axs[0].set_xlabel('l (deg)')
axs[0].set_ylabel('b (deg)')
axs[0].set_xlim(158, 168)
axs[0].set_ylim(-13, -4)


new_reddening = verg_reddening[dustmaps_inds[:, 1].T, dustmaps_inds[:, 0], dustmaps_inds[:, 2]]
norm = matplotlib.colors.Normalize(0, 2)
points = axs[0].scatter(CA_meta['GLON'], CA_meta['GLAT'], c=  CA_meta['Dustmap_AV'], s = 12, cmap = 'viridis', norm = norm, label = '$\int$ Vergely AV')
# fig.colorbar(points, label = 'Integrated AV (l, b) flipped', )
axs[0].legend()


#### 
# axs[1].imshow(np.sum(vergely * 5, axis = 2), origin = 'lower', cmap = 'binary', extent = (l0-9, l0+9, b0-9, b0+9))
axs[1].imshow(bayestar[:, :, -1], origin = 'lower', cmap = 'binary', extent = (l0-9, l0+9, b0-9, b0+9))

axs[1].set_xlabel('l (deg)')
axs[1].set_ylabel('b (deg)')
axs[1].set_xlim(158, 168)
axs[1].set_ylim(-13, -4)


new_reddening = verg_reddening[dustmaps_inds[:, 1], dustmaps_inds[:, 0], dustmaps_inds[:, 2]]
norm = matplotlib.colors.Normalize(0, 2)
points = axs[1].scatter(CA_meta['GLON'], CA_meta['GLAT'], c=  CA_meta['VERG_AV'], s = 12, cmap = 'viridis', norm = norm, label = 'G-TOMO AV')
fig.colorbar(points, label = 'AV', fraction=0.046, pad=0.04)
fig.set_facecolor('white')

axs[1].legend()
plt.show()



fig, ax = plt.subplots(figsize = (8,8))
ax.imshow(bayestar[:, :, -1], origin = 'lower', cmap = 'binary', extent = (l0-9, l0+9, b0-9, b0+9))
ax.set_xlabel('l (deg)')
ax.set_ylabel('b (deg)')
ax.set_xlim(158, 168)
ax.set_ylim(-13, -4)

norm = matplotlib.colors.Normalize(0, 3)
points = ax.scatter(CA_meta['GLON'], CA_meta['GLAT'], c= CA_meta['Dustmap_AV'], s = 12, cmap = 'viridis', norm = norm)
fig.colorbar(points, label = 'Vergely Per-Star AV', )
fig.set_facecolor('white')

plt.show()

fig, ax = plt.subplots(figsize = (8,8))
ax.imshow(bayestar[:, :, -1], origin = 'lower', cmap = 'binary', extent = (l0-9, l0+9, b0-9, b0+9))
ax.set_xlabel('l (deg)')
ax.set_ylabel('b (deg)')
ax.set_xlim(158, 168)
ax.set_ylim(-13, -4)

norm = matplotlib.colors.Normalize(0, 3)
points = ax.scatter(CA_meta['GLON'], CA_meta['GLAT'], c= bayestar_array, s = 12, cmap = 'viridis', norm = norm)
fig.colorbar(points, label = 'Bayestar Per-Star AV', )
fig.set_facecolor('white')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(bayestar_array, new_reddening, alpha = 0.6)
ax.set_xlim(0, 3)
ax.set_ylim(0,3)
ax.set_aspect('equal')
ax.set_xlabel('Bayestar A(V)')
ax.set_ylabel('Verg per-star A(V)')
ax.plot([0, 3], [0, 3], linestyle = 'dashed', color = 'r')

plt.show()

fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(bayestar_array, CA_meta['Dustmap_AV'], alpha = 0.6)
ax.set_xlim(0, 3)
ax.set_ylim(0,3)
ax.set_aspect('equal')
ax.set_xlabel('Bayestar A(V)')
ax.set_ylabel('Vergely Integrated A(V)')
ax.plot([0, 3], [0, 3], linestyle = 'dashed', color = 'r')

plt.show()

fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(CA_meta['AV50'], bayestar_array, alpha = 0.6)
ax.set_xlim(0, 3)
ax.set_ylim(0,3)
ax.set_aspect('equal')
ax.set_xlabel('StarHorse AV')
ax.set_ylabel('Bayestar AV')
ax.plot([0, 3], [0, 3], linestyle = 'dashed', color = 'r')

plt.show()

fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(CA_meta['AV50'], verg_array, alpha = 0.6)
ax.set_xlim(0, 3)
ax.set_ylim(0,3)
ax.set_aspect('equal')
ax.set_xlabel('StarHorse AV')
ax.set_ylabel('Verg per-star AV')
ax.plot([0, 3], [0, 3], linestyle = 'dashed', color = 'r')

plt.show()


fig, ax = plt.subplots(figsize = (8,8))
ax.scatter(CA_meta['AV50'], CA_meta['Dustmap_AV'], alpha = 0.6)
ax.set_xlim(0, 3)
ax.set_aspect('equal')
ax.set_ylim(0,3)
ax.set_xlabel('StarHorse AV')
ax.set_ylabel('Verg AV (integrated)')
ax.plot([0, 3], [0, 3], linestyle = 'dashed', color = 'r')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))
ax.scatter(CA_meta['VERG_AV'], new_reddening, alpha = 0.6)
ax.plot([0.5, 2.5],[0.5, 2.5], linestyle = 'dashed', color = 'red')
ax.set_xlabel('G-TOMO AV')
ax.set_ylabel('Integrated AV - (l,b) flipped')
fig.set_facecolor('white')
plt.show()

fig, ax = plt.subplots(figsize = (8,6))
ax.scatter(CA_meta['VERG_AV'], CA_meta['Dustmap_AV'], alpha = 0.6)
ax.plot([0.5, 2.5],[0.5, 2.5], linestyle = 'dashed', color = 'red')
ax.set_xlabel('G-TOMO AV')
ax.set_ylabel('Integrated AV')
fig.set_facecolor('white')
plt.show()


fig, ax = plt.subplots(figsize = (8,6))
ax.scatter(CA_meta['AV50'], CA_meta['Dustmap_AV'], alpha = 0.6)
ax.plot([0.5, 2.5],[0.5, 2.5], linestyle = 'dashed', color = 'red')
ax.set_xlabel('StarHorse AV')
ax.set_ylabel('Integrated AV')
fig.set_facecolor('white')
plt.show()

fig, ax = plt.subplots(figsize = (8,6))
ax.scatter(CA_meta['AV50'], new_reddening, alpha = 0.6)
ax.plot([0.5, 2.5],[0.5, 2.5], linestyle = 'dashed', color = 'red')
ax.set_xlabel('StarHorse AV')
ax.set_ylabel('Integrated AV (l,b) flipped')
fig.set_facecolor('white')
plt.show()

fig, ax = plt.subplots(figsize = (8,6))
ax.scatter(CA_meta['AV50'], CA_meta['VERG_AV'], alpha = 0.6)
ax.plot([0.5, 2.5],[0.5, 2.5], linestyle = 'dashed', color = 'red')
ax.set_xlabel('StarHorse A(V)')
ax.set_ylabel('G-TOMO A(V)')
fig.set_facecolor('white')
plt.show()

In [ ]:


fig, ax = plt.subplots(figsize = (8,8))
ax.imshow(np.sum(vergely * 5, axis = 2), origin = 'lower', cmap = 'binary', extent = (l0-9, l0+9, b0-9, b0+9))
ax.set_xlabel('l (deg)')
ax.set_ylabel('b (deg)')
ax.set_xlim(158, 168)
ax.set_ylim(-13, -4)

norm = matplotlib.colors.Normalize(-2, 2)
points = ax.scatter(CA_meta['GLON'], CA_meta['GLAT'], c= CA_meta['AK_TARG'] / 0.302 * 2.742 - CA_meta['AV50'], s = 12, cmap = 'coolwarm', norm = norm)
fig.colorbar(points, label = 'AK_TARG AV - Starhorse AV', )

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))
ax.scatter(verg_array, new_reddening)
ax.set_xlim(0, 3)
ax.set_ylim(0,3)
ax.set_xlabel('Vergely Per-Star A(V)')
ax.set_ylabel('Vergely Integrated (l, b) flipped A(V)')
ax.plot([0,3], [0,3], linestyle = 'dashed', c = 'r')
fig.set_facecolor('white')
plt.show()

In [ ]:
fig, ax= plt.subplots()
mguess = 1/(np.sqrt(2 * np.pi) * sigma0) * 102e-3

# line = lambda x, m, b : m * x + b
# line_param = curve_fit(line, AV, amplitude, p0 = [1/(np.sqrt(2 * np.pi) * 1.5) * 102e-3 * 0.5, 0.05], bounds = ((-0.1, -0.05), (.5, .05)), check_finite = False)
select_fit = (np.isnan(amplitude) == False) & (np.isnan(amplitude_errs) == False)
amplitude_fit = np.copy(amplitude)[select_fit]
amplitude_fit_errs = np.copy(amplitude_errs)[select_fit]

line_param = curve_fit(line, CA_meta['AV50'][select_fit], amplitude_fit, p0 = [mguess,0], bounds = ((0, -0.0001),(0.1, 0.000)), check_finite= False, sigma = amplitude_fit_errs)#bounds = ((-0.1, -0.05), (.5, .05))

ax.scatter(CA_meta['AV50'],  amplitude, alpha = 0.5)
# plot versus starhorse and AK_TARG

xx = np.linspace(0,3)
ax.plot(xx, line(xx, *line_param[0]))

ax.plot(xx, line(xx, mguess, 0), label = 'Zasowski+ (2015)', linestyle = 'dashed')
ax.legend()

# ax.set_xlim(0, 3)
plt.show()

In [ ]:
print(line_param)
print(mguess)

In [ ]:
fig, ax = plt.subplots()
ax.hist(fit_params[:, 1, 0], bins = 20)

sig_med = np.nanmedian(fit_params[:, 1, 0])

ymin, ymax = ax.get_ylim()
ax.set_ylim(ymin, ymax)
ax.plot([sig_med, sig_med], [ymin, ymax], linestyle = 'dashed', label = 'med')
ax.legend()
ax.set_xlabel('DIB $\sigma$')
plt.show()
